In [1]:
#!/usr/bin/env python
# coding: utf-8

# data preparation

import os
import pandas as pd
from glob import glob
import numpy as np


from tensorflow.keras import backend
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from keras import models
from keras.layers.advanced_activations import LeakyReLU

from tensorflow.keras.optimizers import Adam
import keras.backend as K
import librosa
import librosa.display
import pylab
import matplotlib.pyplot as plt
from matplotlib import figure
import gc
from path import Path
from keras_preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
        r'D:\reposetory\Save_Transport\dataset\train',
        target_size=(150, 150),
        batch_size=20)

valik = test_datagen.flow_from_directory(
        r'D:\reposetory\Save_Transport\dataset\test',
        target_size=(150, 150),
        batch_size=1
        )


In [ ]:
#!/usr/bin/env python
# coding: utf-8

# model


import tensorflow as tf
import keras.backend as K
def weighted_binary_crossentropy(y_true, y_pred):
    weights = (tf.math.abs(y_true-1) * 59.) + 1.
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    weighted_bce = K.mean(bce * weights)
    return weighted_bce

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), input_shape=(150, 150, 3)),
    tf.keras.layers.Conv2D(16, (3, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.LeakyReLU(alpha=0.3),
    tf.keras.layers.Conv2D(32, (3, 3)),
    tf.keras.layers.Conv2D(32, (3, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.LeakyReLU(alpha=0.3),
    tf.keras.layers.Conv2D(64, (3, 3)),
    tf.keras.layers.Conv2D(64, (3, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.LeakyReLU(alpha=0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512),
    tf.keras.layers.LeakyReLU(alpha=0.3),
    tf.keras.layers.Dense(2, activation='sigmoid')
])


# Настраиваем гиперпараметры нейронной сети
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Начинаем обучение
history = model.fit(train_generator,
                      validation_data=valik,
                      steps_per_epoch=24,
                      epochs=10,
                      validation_steps=3,
                      verbose=2)

model.save('save_1.h5')



Epoch 1/10
24/24 - 9s - loss: 0.4603 - accuracy: 0.8214 - val_loss: 0.4809 - val_accuracy: 0.6667
Epoch 2/10
24/24 - 9s - loss: 0.2794 - accuracy: 0.8382 - val_loss: 0.3601 - val_accuracy: 0.6667
Epoch 3/10
24/24 - 9s - loss: 0.2976 - accuracy: 0.8125 - val_loss: 0.1714 - val_accuracy: 1.0000
Epoch 4/10
24/24 - 10s - loss: 0.2768 - accuracy: 0.8167 - val_loss: 0.2381 - val_accuracy: 1.0000
Epoch 5/10
24/24 - 9s - loss: 0.2466 - accuracy: 0.8571 - val_loss: 0.0988 - val_accuracy: 1.0000
Epoch 6/10
24/24 - 9s - loss: 0.2611 - accuracy: 0.8354 - val_loss: 0.4707 - val_accuracy: 0.6667
Epoch 7/10
24/24 - 8s - loss: 0.2729 - accuracy: 0.8298 - val_loss: 0.2412 - val_accuracy: 1.0000


In [3]:
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
#from data_prep import valik
import pandas as pd
from sklearn.metrics import confusion_matrix
from PIL import Image, ImageFont, ImageDraw

In [4]:
model = load_model(r'D:\reposetory\Save_Transport\main_classifier\save_1.h5')


results = model.evaluate(valik)
print('\n loss, accuracy =', results, '\n')


filenames = valik.filenames
count_samples = len(filenames)
predict = model.predict_generator(valik,steps = count_samples)



#final_res = model.predict(valik[1][0])
#print(final_res)

#data = {'y_Actual':    [ tuple(i) for i in valik[1][1]],
#        'y_Predicted': [ tuple(i) for i in final_res]
#        }

#df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
#print (df)


OSError: SavedModel file does not exist at: D:\reposetory\Save_Transport\main_classifier\save_1.h5/{saved_model.pbtxt|saved_model.pb}

# Confusion Matrix

In [1]:
import math
import random
random.seed(228)
import pandas as pd

In [169]:
model = load_model('/home/aigaf/Downloads/Telegram Desktop/save_1.h5')

In [212]:
generator = ImageDataGenerator(rescale=1./255)
predict_generator = generator.flow_from_directory('/home/aigaf/Desktop/GitHub/Save_Transport/dataset/test',
                                               target_size=(150, 150),  shuffle = True, batch_size = 30)

y_test_1 = model.predict(predict_generator[0][0])
y_test_2 = model.predict(predict_generator[1][0])

Found 60 images belonging to 2 classes.


In [214]:
y_test = []
y_test.extend(y_test_1)
y_test.extend(y_test_2)

y_test =np.argmax(y_test, axis=1)

ValueError: in user code:

    /home/aigaf/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1478 predict_function  *
        return step_function(self, iterator)
    /home/aigaf/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/aigaf/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/aigaf/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/aigaf/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/aigaf/anaconda3/lib/python3.7/site-packages/tensorflow/pyth

In [211]:
y_true_1 = predict_generator[0][1]
y_true_2 = predict_generator[1][1]

y_true = []
y_true.extend(y_true_1) 
y_true.extend(y_test_2)

y_true = np.argmax(y_true, axis=1)

{'y_Actual': [(0.0, 1.0),
  (0.0, 1.0),
  (0.0, 1.0),
  (1.0, 0.0),
  (0.0, 1.0),
  (1.0, 0.0),
  (1.0, 0.0),
  (1.0, 0.0),
  (1.0, 0.0),
  (0.0, 1.0),
  (1.0, 0.0),
  (0.0, 1.0),
  (1.0, 0.0),
  (1.0, 0.0),
  (0.0, 1.0),
  (1.0, 0.0),
  (1.0, 0.0),
  (1.0, 0.0),
  (0.0, 1.0),
  (0.0, 1.0)],
 'y_Predicted': [(0.00017768145, 0.99981445),
  (0.009906739, 0.9904443),
  (0.009906739, 0.9904443),
  (0.52341264, 0.43952236),
  (0.0015658736, 0.9983101),
  (0.6737475, 0.3092239),
  (0.55643094, 0.41479164),
  (0.54197824, 0.4316025),
  (0.5726254, 0.41226786),
  (0.0017089844, 0.9980612),
  (0.59927857, 0.37441114),
  (0.0033317804, 0.99637914),
  (0.5972071, 0.3737676),
  (0.63301736, 0.34296703),
  (0.0031929314, 0.9966687),
  (0.6011083, 0.36867094),
  (0.5131882, 0.44904092),
  (0.5234554, 0.45037204),
  (0.0041032434, 0.9954548),
  (0.0017437935, 0.9980533)]}

In [193]:
data = {'y_Actual':    [ y_true],
        'y_Predicted': [ y_test]
        }

In [194]:
data

In [195]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_test)